In [ ]:
import syft as sy
import pandas as pd
import numpy as np
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey
from syft.core.adp.entity import Entity

In [ ]:
data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv")

# Get canada trade data and convert into unit millions
canada_trade = (np.array(list(data['Trade Value (US$)'])) / 1000000)[0:10]
trade_partners = ((list(data['Partner'])))[0:10]

data_batch = canada_trade
data_batch = (data_batch * 10).astype('int32')

entities = list()
for i in range(len(data_batch)):
    entities.append(Entity(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
x = sy.Tensor(data_batch)#.private(0, 2, entity=entities[0]).tag("trade_flow")
x.public_shape = x.shape

y = sy.Tensor(data_batch)#.private(0, 2, entity=entities[0]).tag("trade_flow")
y.public_shape = y.shape

In [ ]:
def encode_key(key: SigningKey) -> str:
    return key.encode(encoder=HexEncoder).decode("utf-8")

_alice = sy.Domain("alice")
alice = _alice.get_root_client()

alice_key = SigningKey.generate()
_alice.users.signup(
    name="Bob",
    email="bob@gmail.com",
    password="letmein",
    budget=10000000000,
    role=1,
    private_key=encode_key(alice_key),
    verify_key=encode_key(alice_key.verify_key),
)
alice.verify_key = alice_key.verify_key
alice.signing_key = alice_key

_bob = sy.Domain("bob")
bob = _bob.get_root_client()

bob_key = SigningKey.generate()
_bob.users.signup(
    name="Bob",
    email="bob@gmail.com",
    password="letmein",
    budget=10000000000,
    role=1,
    private_key=encode_key(bob_key),
    verify_key=encode_key(bob_key.verify_key),
)
bob.verify_key = bob_key.verify_key
bob.signing_key = bob_key

In [ ]:
xp = x.send(alice)
yp = y.send(bob)

dp_x = x.private(0, 2, entity=entities[0]).tag("trade_flow")

In [ ]:
dp_x

In [ ]:
dp_x_p = dp_x.send(alice)

In [ ]:
type(dp_x_p), dp_x_p.id_at_location

In [ ]:
out = dp_x_p.share(bob)

In [ ]:
type(out), [child.id_at_location for child in out.child]

In [ ]:
# out2 = out.publish(sigma=0.01)
out2 = out.publish(sigma=0.1)

In [ ]:
type(out2), [child.id_at_location for child in out.child]

In [ ]:
f = out2.get()

In [ ]:
type(f)

In [ ]:
f

In [ ]:
x